# Convolutional Neural Network (CNN)

Good ConvNets are beasts with millions of parameters and many hidden layers. In fact, a bad rule of thumb is: ‘higher the number of hidden layers, better the network’. AlexNet, VGG, Inception, ResNet are some of the popular networks. Why do these networks work so well? How are they designed? Why do they have the structures they have? One wonders. The answer to these questions is not trivial and certainly, can’t be covered in one lecture. However, in this session , we shall try to discuss some of these questions. Network architecture design is a complicated process and will take a while to learn and even longer to experiment designing on your own. Lets start by remembering the concepts of image classification and feature extraction 

# **Image classification**

Image classification is the task of classifying a given image into one of the pre-defined categories. Traditional pipeline for image classification involves two modules: viz. feature extraction and classification. 

**Feature extraction**

Feature extraction involves extracting a higher level of information from raw pixel values that can capture the distinction among the categories involved. This feature extraction is done in an unsupervised manner wherein the classes of the image have nothing to do with information extracted from pixels. Some of the traditional and widely used features are GIST, HOG, SIFT, LBP etc. After the feature is extracted, a classification module is trained with the images and their associated labels. A few examples of this module are SVM, Logistic Regression, Random Forest, decision trees etc.

Lets also remember some of the most design aspects to consider when proposing an architecture: accuracy and computation.

# **Accuracy**

If you are building an intelligent machine, it is absolutely critical that it must be as accurate as possible. One fair question to ask here is that ‘accuracy not only depends on the network but also on the amount of data available for training’. Hence, these networks are compared on a standard dataset called ImageNet.

ImageNet project is an ongoing effort and currently has 14,197,122 images from 21841 different categories. Since 2010, ImageNet has been running an annual competition in visual recognition where participants are provided with 1.2 million images belonging to 1000 different classes from Imagenet data-set. So, each network architecture reports accuracy using these 1.2 million images of 1000 classes.

# **Computation**

Most ConvNets have huge memory and computation requirements, especially while training. Hence, this becomes an important concern. Similarly, the size of the final trained model becomes important to consider if you are looking to deploy a model to run locally on mobile. As you can guess, it takes a more computationally intensive network to produce more accuracy. So, there is always a trade-off between accuracy and computation.

Apart from these, there are many other factors like ease of training, the ability of a network to generalize well etc. The networks described below are the most popular ones and are presented in the order that they were published and also had increasingly better accuracy from the earlier ones.

# **AlexNet**

This architecture was one of the first deep networks to push ImageNet Classification accuracy by a significant stride in comparison to traditional methodologies. It is composed of 5 convolutional layers followed by 3 fully connected layers.

![alt text](https://www.researchgate.net/publication/320052364/figure/fig1/AS:543136445198336@1506505227088/Scheme-of-the-AlexNet-network-used.png)

AlexNet, proposed by Alex Krizhevsky, uses ReLu(Rectified Linear Unit) for the non-linear part, instead of a Tanh or Sigmoid function which was the earlier standard for traditional neural networks. ReLu is given by 

f(x) = max(0,x)

The advantage of the ReLu over sigmoid is that it trains much faster than the latter because the derivative of sigmoid becomes very small in the saturating region and therefore the updates to the weights almost vanish(Figure 4). This is called vanishing gradient problem.

In the network, ReLu layer is put after each and every convolutional and fully-connected layers(FC).



# **VGG16**

This architecture is from VGG group, Oxford. It makes the improvement over AlexNet by replacing large kernel-sized filters(11 and 5 in the first and second convolutional layer, respectively) with multiple 3X3 kernel-sized filters one after another. With a given receptive field(the effective area size of input image on which output depends), multiple stacked smaller size kernel is better than the one with a larger size kernel because multiple non-linear layers increases the depth of the network which enables it to learn more complex features, and that too at a lower cost. 

For example, three 3X3 filters on top of each other with stride 1 ha a receptive size of 7, but the number of parameters involved is 3*(9C^2) in comparison to 49C^2 parameters of kernels with a size of 7. Here, it is assumed that the number of input and output channel of layers is C.Also, 3X3 kernels help in retaining finer level properties of the image. The network architecture is given in the table.

![alt text](https://neurohive.io/wp-content/uploads/2018/11/vgg16-1-e1542731207177.png)

You can see that in VGG-D, there are blocks with same filter size applied multiple times to extract more complex and representative features. This concept of blocks/modules became a common theme in the networks after VGG.

The VGG convolutional layers are followed by 3 fully connected layers. The width of the network starts at a small value of 64 and increases by a factor of 2 after every sub-sampling/pooling layer. It achieves the top-5 accuracy of 92.3 % on ImageNet.


# **Inception**

While VGG achieves a phenomenal accuracy on ImageNet dataset, its deployment on even the most modest sized GPUs is a problem because of huge computational requirements, both in terms of memory and time. It becomes inefficient due to large width of convolutional layers.

For instance, a convolutional layer with 3X3 kernel size which takes 512 channels as input and outputs 512 channels, the order of calculations is 9X512X512. 

In a convolutional operation at one location, every output channel (512 in the example above), is connected to every input channel, and so we call it a dense connection architecture. The Inception builds on the idea that most of the activations in a deep network are either unnecessary(value of zero) or redundant because of correlations between them. Therefore the most efficient architecture of a deep network will have a sparse connection between the activations, which implies that all 512 output channels will not have a connection with all the 512 input channels. There are techniques to prune out such connections which would result in a sparse weight/connection. But kernels for sparse matrix multiplication are not optimized in BLAS or CuBlas(CUDA for GPU) packages which render them to be even slower than their dense counterparts.

So Inception devised a module called inception module that approximates a sparse CNN with a normal dense construction(shown in the figure). Since only a small number of neurons are effective as mentioned earlier, the width/number of the convolutional filters of a particular kernel size is kept small. Also, it uses convolutions of different sizes to capture details at varied scales(5X5, 3X3, 1X1).

Another salient point about the module is that it has a so-called bottleneck layer(1X1 convolutions in the figure). It helps in the massive reduction of the computation requirement as explained below.

Let us take the first inception module of Inception as an example which has 192 channels as input. It has just 128 filters of 3X3 kernel size and 32 filters of 5X5 size. The order of computation for 5X5 filters is 25X32X192 which can blow up as we go deeper into the network when the width of the network and the number of 5X5 filter further increases. In order to avoid this, the inception module uses 1X1 convolutions before applying larger sized kernels to reduce the dimension of the input channels, before feeding into those convolutions. So in the first inception module, the input to the module is first fed into 1X1 convolutions with just 16 filters before it is fed into 5X5 convolutions. This reduces the computations to 16X192 +  25X32X16. All these changes allow the network to have a large width and depth.

Another change that Inception made, was to replace the fully-connected layers at the end with a simple global average pooling which averages out the channel values across the 2D feature map, after the last convolutional layer. This drastically reduces the total number of parameters. This can be understood from AlexNet, where FC layers contain approx. 90% of parameters. Use of a large network width and depth allows Inception to remove the FC layers without affecting the accuracy. It achieves 93.3% top-5 accuracy on ImageNet and is much faster than VGG.

![alt text](https://miro.medium.com/max/960/1*gqKM5V-uo2sMFFPDS84yJw.png)

# **RESNET**

As per what we have seen so far, increasing the depth should increase the accuracy of the network, as long as over-fitting is taken care of. But the problem with increased depth is that the signal required to change the weights, which arises from the end of the network by comparing ground-truth and prediction becomes very small at the earlier layers, because of increased depth. It essentially means that earlier layers are almost negligible learned. This is called vanishing gradient. The second problem with training the deeper networks is, performing the optimization on huge parameter space and therefore naively adding the layers leading to higher training error. Residual networks allow training of such deep networks by constructing the network through modules called residual models as shown in the figure. This is called degradation problem. The intuition around why it works can be seen as follows:

Imagine a network, A which produces x amount of training error. Construct a network B by adding few layers on top of A and put parameter values in those layers in such a way that they do nothing to the outputs from A. Let’s call the additional layer as C. This would mean the same x amount of training error for the new network. So while training network B, the training error should not be above the training error of A. And since it DOES happen, the only reason is that learning the identity mapping(doing nothing to inputs and just copying as it is) with the added layers-C is not a trivial problem, which the solver does not achieve. To solve this, the module shown above creates a direct path between the input and output to the module implying an identity mapping and the added layer-C just need to learn the features on top of already available input. Since C is learning only the residual, the whole module is called residual module. 

Also, similar to Inception, it uses a global average pooling followed by the classification layer. Through the changes mentioned, ResNets were learned with network depth of as large as 152. It achieves better accuracy than VGGNet and Inception while being computationally more efficient than VGGNet. ResNet-152 achieves 95.51 top-5 accuracies. 

The architecture is similar to the VGGNet consisting mostly of 3X3 filters. From the VGGNet, shortcut connection as described above is inserted to form a residual network. This can be seen in the figure which shows a small snippet of earlier layer synthesis from VGG-19.

The power of the residual networks can be judged from one of the experiments in paper 4. The plain 34 layer network had higher validation error than the 18 layers plain network. This is where we realize the degradation problem. And the same 34 layer network when converted into the residual network has much lesser training error than the 18 layer residual network. 

![alt text](https://1.bp.blogspot.com/-oteqcXt92Dk/XSab__AVdGI/AAAAAAAAE-c/aDFbRmaq6Pw0uUB2BzDXtnqiokg0wswtQCLcBGAs/s1600/1_S3TlG0XpQZSIpoDIUCQ0RQ.png)

**Lets get our hands into the real stuff!**

The training we will practice is very similar to our first CNN practice, cifar 10 dataset but instead of using a standard and simple CNN we will practice 2 of the above mentioned "super models": VGG and Resnet.

# Example

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
import numpy
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

### Download and prepare the CIFAR10 dataset


The CIFAR10 dataset contains 60,000 color images in 10 classes, with 6,000 images in each class. The dataset is divided into 50,000 training images and 10,000 testing images. The classes are mutually exclusive and there is no overlap between them.

In [ ]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.vgg16 import preprocess_input
(Raw_train_images, train_labels), (Raw_test_images, test_labels) = datasets.cifar10.load_data()
plt.imshow(Raw_train_images[1])
print(train_labels[1])
# Normalize pixel values to be between 0 and 1

train_images = preprocess_input(Raw_train_images)
test_images = preprocess_input(Raw_test_images)
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

### Verify the data

To verify that the dataset looks correct, let's plot the first 25 images from the training set and display the class name below each image.


In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(True)
    plt.imshow(Raw_train_images[i], cmap=plt.cm.binary)
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    plt.xlabel(class_names[numpy.argmax(train_labels[i])])
plt.show()

**Create the convolutional base**

The next code segment will construct the model according t othe MODE variable, the model setup varies from desing to another but generally it has the input size, the number of classes and the intial weights (we will set it to none in this lecture).

In [ ]:
MODE = 'RESNET' # VGG or RESNET
if MODE =='VGG':
  from tensorflow.keras.applications.vgg16 import VGG16
  from tensorflow.keras.callbacks import ModelCheckpoint
  model = VGG16(
      weights=None, 
      include_top=True, 
      classes=10,
      input_shape=(32,32,3)
  )
elif MODE =='RESNET':
  from keras.applications.resnet50 import ResNet50
  from keras.models import Model
  from keras.layers import Dense
  from keras.layers import Flatten, Dropout
  from keras.callbacks import ModelCheckpoint
  from keras import Sequential
  # load model without classifier layers
  Base_model = ResNet50(include_top=False, input_shape=(32, 32, 3))
  model = Sequential()
  #Add the Dense layers along with activation and batch normalization
  model.add(Base_model)
  model.add(Flatten())


  #Add the Dense layers along with activation and batch normalization
  model.add(Dense(4000,activation=('relu'),input_dim=512))
  model.add(Dense(2000,activation=('relu'))) 
  model.add(Dropout(.4))
  model.add(Dense(1000,activation=('relu'))) 
  model.add(Dropout(.3))#Adding a dropout layer that will randomly drop 30% of the weights
  model.add(Dense(500,activation=('relu')))
  model.add(Dropout(.2))
  model.add(Dense(10,activation=('softmax')))
else:
  print ('Please set a valid model name')
  raise Exception('Please set a valid model name')

Let's display the architecture of our model so far.

In [ ]:
model.summary()

As for the VGG net, there is no additional modifications are added to the model. While the Resnet which is a much advanced model requires appending a set of dense layers to handle the classification task. Those dense layers are paired with the drop out regularization to make sure we wont overfit during the process.



### Compile and train the model

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='sgd',
    metrics=['accuracy']
)
checkpoint = ModelCheckpoint(
    'model.h5', 
    monitor='val_accuracy', 
    verbose=0, 
    save_best_only=True, 
    save_weights_only=False,
    mode='auto',
)

# Train the model
history = model.fit(
    x=train_images,
    y=train_labels,
    validation_data=(test_images, test_labels),
    validation_split=0.1,
    batch_size=256,
    epochs=100,
    callbacks=[checkpoint],
    verbose=1
)

### Evaluate the model

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

In [ ]:
print(test_acc)

Our selected CNN has achieved a test accuracy better than our first simple CNN. That is a big improvmenet and probably applicable for other problems! Away from numbers, lets try to evaluate the model inference abilities.



In [ ]:
import random,numpy
iEvalIx= random.randrange(len(test_labels)-1)
iClassIX =test_labels[iEvalIx][0]
plt.imshow(Raw_test_images[iEvalIx], cmap=plt.cm.binary)
iPredictionIx = numpy.argmax(model.predict(test_images[iEvalIx].reshape(1,32,32,3)))
print ("I think this is a", class_names[iPredictionIx]) 